In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('csv/processed_tickets_en.csv')
df.shape

(1564, 5)

In [3]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for each sentence
def get_bert_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        # Tokenize the sentence
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)
        
        # Get the BERT embeddings (output of the last hidden layer)
        with torch.no_grad():
            outputs = bert_model(**inputs)
            last_hidden_state = outputs.last_hidden_state  # shape: [batch_size, seq_length, hidden_size]
        
        # Use the [CLS] token's embedding as the sentence representation
        cls_embedding = last_hidden_state[:, 0, :].numpy()
        embeddings.append(cls_embedding)
    
    return np.vstack(embeddings)

# Example usage: Assume `data` is your list of ticket descriptions
data = df['processed_text'].tolist()

# Get BERT embeddings
X_bert = get_bert_embeddings(data)


/Users/kripa/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [88]:
from sklearn.model_selection import train_test_split
X = df['processed_text']
y = df['urgency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
 
# Load BERT tokenizer and model
model_name = "bert-base-uncased"  # You can replace this with other models like 'roberta-base' or 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
 
# Function to tokenize and generate BERT embeddings
def bert_vectorize(texts):
    """
    Converts a list of texts into BERT embeddings.
    :param texts: List of text strings
    :return: Numpy array of embeddings
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the token embeddings (pooled output) for the final sentence representation
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()
 
# Convert your dataset to BERT embeddings
X_train_bert = bert_vectorize(X_train.values.tolist())
X_test_bert = bert_vectorize(X_test.values.tolist())
 
# Check the shapes of the resulting arrays
print("X_train_bert shape:", X_train_bert.shape)
print("X_test_bert shape:", X_test_bert.shape)
 
 
# Load BERT tokenizer and model
model_name = "bert-base-uncased"  # You can replace this with other models like 'roberta-base' or 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
 
# Function to tokenize and generate BERT embeddings
def bert_vectorize(texts):
    """
    Converts a list of texts into BERT embeddings.
    :param texts: List of text strings
    :return: Numpy array of embeddings
    """
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the token embeddings (pooled output) for the final sentence representation
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()
 
# Convert your dataset to BERT embeddings
X_train_bert = bert_vectorize(X_train.values.tolist())
X_test_bert = bert_vectorize(X_test.values.tolist())
 
# Check the shapes of the resulting arrays
print("X_train_bert shape:", X_train_bert.shape)
print("X_test_bert shape:", X_test_bert.shape)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Example of adding TF-IDF features (you can skip this if you already have them)
vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(data).toarray()

# Combine BERT embeddings with TF-IDF features
X_combined = np.hstack([X_tfidf, X_bert])


In [6]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

y = df['urgency'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# Make predictions
y_pred_logreg = logreg_model.predict(X_test)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logreg))

# Train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Make predictions
y_pred_svm = svm_model.predict(X_test)
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

  not_urgent       0.96      0.93      0.94       156
      urgent       0.93      0.96      0.95       157

    accuracy                           0.95       313
   macro avg       0.95      0.95      0.95       313
weighted avg       0.95      0.95      0.95       313

SVM Classification Report:
              precision    recall  f1-score   support

  not_urgent       1.00      0.97      0.98       156
      urgent       0.97      1.00      0.98       157

    accuracy                           0.98       313
   macro avg       0.98      0.98      0.98       313
weighted avg       0.98      0.98      0.98       313



In [83]:
# Example prediction
# New example text to test the model
example_text = [
    "we need to find the dataset ourselves based on the use case",
    "Are we instructed anything on POC?",
    "I am not able to access the data in the shared folder",
    "nicee emergency and non emergency",
    "rasied pr for bug fixes. let me know when it is approved. no hurry",
    "once everything works fine we will push to stage"
]

# Get BERT embeddings
X_bert_example = get_bert_embeddings(example_text)

# Get TF-IDF features
X_tfidf_example = vectorizer.transform(example_text).toarray()

# Combine BERT embeddings with TF-IDF features
X_combined_example = np.hstack([X_tfidf_example, X_bert_example])


In [84]:
# make prediction
# Predict with Logistic Regression model
y_pred_logreg_example = logreg_model.predict(X_combined_example)
print("Logistic Regression Prediction :", y_pred_logreg_example)

Logistic Regression Prediction : ['not_urgent' 'not_urgent' 'not_urgent' 'not_urgent' 'urgent' 'not_urgent']


In [85]:
y_pred_svm_example = svm_model.predict(X_combined_example)
print("SVM Prediction (0: Non-Urgent, 1: Urgent):", y_pred_svm_example)

SVM Prediction (0: Non-Urgent, 1: Urgent): ['not_urgent' 'not_urgent' 'urgent' 'not_urgent' 'not_urgent' 'not_urgent']
